In [ ]:
from ultralytics import YOLO
from ultralytics import settings
import torch
import pandas as pd
import torch
import os
import glob
import wandb
import ultralytics
import cv2
print('ultralytics %s' % ultralytics.__version__)
print('wandb %s' % wandb.__version__)
from wandb.integration.ultralytics import add_wandb_callback

In [ ]:
!nvidia-smi

Arguments setting for the model

In [3]:
args = dict(
    data = "configs/data.yaml",
    epochs = 1,
    patience = 5,
    batch = 4,
    save = False,
    save_period = 5,
    device = '0',
    workers = 1,
    project = "weights",
    name = "try",
    exist_ok = False, #overwrite the existing run for iterative testing, turn false when confident
    pretrained = True,
    optimizer = 'auto',
    resume = False,
    amp = False, #mixed precision training for faster training, might cause some issues
    fraction = 0.1, #fraction of dataset if resouce limited, change to 1 when confident
    profile = False, #enable ONNX and TensorRT profiling
    freeze = None, #freeze the first N layers or can specify the layers by index
    #for validation part
    conf = 0.01,
    iou = 0.6,
    degrees = 0.0, 
    shear = 0.0,
    perspective = 0.0,
    plots = True,
    half = True, #use half precision for weaker gpu
)

Loading the model yoloV8 and running wandb for logging

Training of model

In [ ]:
settings.update({"wandb": True})
wandb.login()

In [ ]:
wandb.init(project='YOLOv8x-GlottisDetection', name='Parameter-sweep 1')

In [ ]:
# for loading model
model = YOLO('yolov8x.pt')

In [ ]:
add_wandb_callback(model, enable_model_checkpointing=True)

In [ ]:
#training model
results = model.train(cfg="train_cfg.yaml")

In [9]:
model_path = 'yolov8nDemo.pt'
model.save(model_path)

In [14]:

wandb.finish()

Running Predictions on model using bagls video and dhs video

Can use OpenCV for better video processing and visualization of the predictions

In [17]:
dhs_video = r'D:\FYP Code\testfiles\dhs.mp4'
baglsC_video = r'D:\FYP Code\testfiles\bagls-color.mp4'
bagls_video = r'D:\FYP Code\testfiles\bagls.mp4'
test_path = r'D:\FYP Datasets\fyp\training\test\images'
weight_path = r'D:\FYP Code\weights\try2\weights\best.pt'

In [ ]:
model = YOLO(weight_path)
# model.predict(source= test_path, save_dir = 'results', save= True)
model.predict(source=dhs_video, save_dir = 'results', save= True)
model.predict(source=baglsC_video, save_dir = 'results', save= True)
model.predict(source=bagls_video, save_dir = 'results', save= True)


In [ ]:
# wandb.log({"final_mAP": results.metrics.mAP50})
# wandb.finish()